In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
import numpy as np

### Download

In [ ]:
# Загрузим данные, изучим их

train = pd.read_csv('data/train.csv')
print(train.shape)
train.head()

In [ ]:
test = pd.read_csv('data/test.csv')
print(test.shape)
test.head()

In [ ]:
sample = pd.read_csv('data/sample_submission.csv')
print(sample.shape)
sample.head()

In [ ]:
print(sample['oid'].value_counts())
print(test['oid'].value_counts())

In [ ]:
# Объеденим test и sample по iod, чтобы у тестовой выборки была размечена целевая переменная

test = pd.merge(test, sample, on='oid', how='inner')
test.head()

### Preparation

In [ ]:
# Удалим столбец с id, после проверим на дубликаты

train = train.drop(columns=['oid'])
test = test.drop(columns=['oid'])

In [ ]:
print(f'Количество дубликатов в train: {train.duplicated().sum()}')
print(f'Количество дубликатов в test: {test.duplicated().sum()}')

In [ ]:
train = train[~train.duplicated()]
test = test[~test.duplicated()]

print(train.shape)
print(test.shape)

In [ ]:
print(train.info())
print()
print(test.info())

In [ ]:
train['category'].value_counts()

In [ ]:
test['category'].value_counts()

In [ ]:
# Создадим объект LabelEncoder и переведем целевую переменную в числовой формат

le = LabelEncoder()

train['labels'] = le.fit_transform(train['category'])
test['labels'] = le.transform(test['category'])

# Удалим текстовую целевую переменную из данных

train = train.drop(columns=['category'])
test = test.drop(columns=['category'])

print(train.head(3))
print(test.head(3))

In [ ]:
# Создадим словарь соответсвия для целевой переменной

label_names = le.classes_

names_dict = {}

for i in range(len(label_names)):
    names_dict[i] = label_names[i]

names_dict

### Modeling

In [ ]:
# Загрузим модель и токенизатор

model_name = "cointegrated/rubert-tiny"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_names))

In [ ]:
# Переведем данные в необходимый для модели формат

train = Dataset.from_pandas(train)
test = Dataset.from_pandas(test)

In [ ]:
# Создадим функцию и токенизируем данные

def tokenize(batch):
    return tokenizer(batch['text'], max_length=128, padding='max_length', truncation=True)

train = train.map(tokenize, batched=True)
test = test.map(tokenize, batched=True)

In [ ]:
# Функция для отображения метрик

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")
    return {"accuracy": acc, "f1": f1}


# Настроим отображение категорий

model.config.id2label = names_dict
model.config.label2id = {v: k for k, v in model.config.id2label.items()}

In [ ]:
# Аргументы обучения

args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train,
    eval_dataset=test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Точность модели 0.70
Выходи ячеек удалены из-за невозможности отображения виджетов на GitHub. Для ознакомления с полным процессом обучения можно скачать другой файл с таким же названием.

In [ ]:
model.save_pretrained("model/")
tokenizer.save_pretrained("model/")

In [ ]:
from transformers import pipeline

# Проверим работу внутри блокнота
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

result = classifier("Медведев закончил игру подачей навылет. Гейм, сет, матч")
print(result)


In [ ]:
result = classifier("Отличный прыжок нашего спортсмена со снаряда, идеальное приземление")
print(result)